# Semantic Analysis Pipeline

This notebook demonstrates the semantic text analysis capabilities using our custom analyzers.

## Setup
Import required packages and configure the environment:



In [1]:
# At start of notebook
import sys
from pathlib import Path
import logging
import os

# Add project root to Python path
project_root = str(Path().resolve().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# import logging
# from src.nb_helpers.logging import configure_logging

# Set up environment with DEBUG level
from src.nb_helpers.environment import setup_notebook_env, verify_environment
setup_notebook_env(log_level="DEBUG")

# Any verification needed will maintain DEBUG level
verify_environment(log_level="DEBUG")

2024-11-18 12:27:39,699 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:27:39,700 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:39,705 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:27:39,706 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


Environment Check Results:

Basic Setup:
-----------
✓ Project root in path
✓ FileUtils initialized
✓ .env file loaded

Environment Variables:
---------------------
✓ OPENAI_API_KEY set
✓ ANTHROPIC_API_KEY set

Project Structure:
-----------------
✓ Raw data exists
✓ Processed data exists
✓ Configuration exists
✓ Main config.yaml exists

Environment Status: Ready ✓


True

In [3]:
# Import necessary components
from src.loaders.parameter_handler import ParameterHandler
from src.nb_helpers.analyzers import (
    analyze_keywords,
    analyze_themes,
    analyze_categories,
    analyze_text,
    AnalysisOptions
)

from scripts.migrate_parameters import create_example_parameters
from src.nb_helpers.logging import configure_logging, verify_logging_setup_with_hierarchy, reset_debug_logging
from src.loaders.parameter_handler import ParameterHandler, get_parameter_file_path, verify_parameter_file



In [4]:
# Set initial logging
configure_logging(level="DEBUG")
# Keep HTTP loggers at INFO
for name in ["httpx", "httpcore", "openai", "anthropic"]:
    logging.getLogger(name).setLevel(logging.INFO)
    
verify_logging_setup_with_hierarchy()

2024-11-18 12:27:39,843 - src.nb_helpers.logging - DEBUG - Logging configured at DEBUG level



Logging Configuration:
--------------------------------------------------

Logger: root
Set Level: DEBUG
Effective Level: DEBUG
Propagates to root: True
Handlers:
  Handler 1 level: DEBUG

Logger: src.nb_helpers.analyzers
Hierarchy:
  src: NOTSET
  src.nb_helpers: NOTSET
  src.nb_helpers.analyzers: DEBUG
Set Level: DEBUG
Effective Level: DEBUG
Propagates to root: True
No handlers (uses root handlers)

Logger: src.analyzers.keyword_analyzer
Hierarchy:
  src: NOTSET
  src.analyzers: NOTSET
  src.analyzers.keyword_analyzer: DEBUG
Set Level: DEBUG
Effective Level: DEBUG
Propagates to root: True
No handlers (uses root handlers)

Logger: src.analyzers.theme_analyzer
Hierarchy:
  src: NOTSET
  src.analyzers: NOTSET
  src.analyzers.theme_analyzer: DEBUG
Set Level: DEBUG
Effective Level: DEBUG
Propagates to root: True
No handlers (uses root handlers)

Logger: src.analyzers.category_analyzer
Hierarchy:
  src: NOTSET
  src.analyzers: NOTSET
  src.analyzers.category_analyzer: DEBUG
Set Level: DEB

In [5]:
# detailed_logging_info = True
# if detailed_logging_info:
#     from src.nb_helpers.logging import verify_logging_setup_with_hierarchy
#     # Configure logging
#     # configure_logging(level="DEBUG")
#     # Verify with detailed information
#     verify_logging_setup_with_hierarchy()


In [6]:
# Example texts in different languages
example_texts = {
    "English Technical": """
        The cloud migration project improved system scalability while reducing costs.
        New DevOps practices streamlined the deployment pipeline significantly.
    """,
    
    "Finnish Technical": """
        Pilvipalveluihin siirtyminen paransi järjestelmän skaalautuvuutta ja vähensi kustannuksia.
        Uudet DevOps-käytännöt tehostivat merkittävästi käyttöönottoprosessia.
    """,
    
    "English Business": """
        Q3 financial results show 15% revenue growth and improved profit margins.
        Customer acquisition costs decreased while retention rates increased.
    """,
    
    "Finnish Business": """
        Q3 taloudelliset tulokset osoittavat 15% liikevaihdon kasvun ja parantuneet katteet.
        Asiakashankinnan kustannukset laskivat ja asiakaspysyvyys parani.
    """
}



In [7]:
# Create and load parameters
params_file_name = "parameters_en.xlsx"

# Get the full parameter file path
params_file = get_parameter_file_path(params_file_name)

# Create file if it doesn't exist
if not params_file.exists():
    params_file = create_example_parameters(params_file_name)
    print(f"Created parameter file at: {params_file}")
else:
    print(f"Using existing parameter file at: {params_file}")

# Verify the file
verify_parameter_file(params_file)

# Load parameters
handler = ParameterHandler(params_file_name)  # Can now use just the file name
params = handler.get_parameters()



2024-11-18 12:27:39,969 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:27:39,969 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-18 12:27:39,971 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:39,971 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer


Using existing parameter file at: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx

Parameter File Verification:
Absolute path: C:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx
File exists: True


2024-11-18 12:27:41,260 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO



Found sheets:

General Parameters:
                parameter                           value  \
0            max_keywords                              10   
1                focus_on  technical and business content   
2  column_name_to_analyze                         content   
3      min_keyword_length                               3   
4       include_compounds                            True   

                          description  
0  Maximum keywords to extract (1-20)  
1                 Analysis focus area  
2          Name of the content column  
3              Minimum keyword length  
4              Include compound words  

Categories:
              category                                 description  \
0    technical_content  Technical and software development content   
1     business_content              Business and financial content   
2  educational_content            Educational and training content   

                                            keywords  threshold

2024-11-18 12:27:41,260 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-18 12:27:41,263 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:41,263 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:41,272 - src.loaders.parameter_handler - DEBUG - Using parameter file: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\parameters\parameters_en.xlsx
2024-11-18 12:27:41,274 - src.loaders.parameter_handler - DEBUG - Loading Excel file from: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\s

In [8]:
print("\nLoaded parameters:")
# params.print()  # Uses the new print method

# Or just
print(params)  # Uses the new __str__ method


Loaded parameters:
ParameterSet(
  general=max_keywords=10 min_keyword_length=3 language='en' focus_on='technical and business content' include_compounds=True max_themes=3 min_confidence=0.3 column_name_to_analyze='content',
  categories=3 items,
  predefined_keywords=3 items,
  excluded_keywords=3 items,
  analysis_settings=theme_analysis=ThemeAnalysisSettings(enabled=True, min_confidence=0.5) weights=AnalysisWeights(statistical=0.4, llm=0.6),
  domain_context=0 items
)


In [9]:
# Create example files

create_new_example_files=False

if create_new_example_files:
    en_params = create_example_parameters("parameters_en.xlsx", "en")
    fi_params = create_example_parameters("parameters_fi.xlsx", "fi")
    print(f"Created parameter files:\n- {en_params}\n- {fi_params}")
    print(f"Created parameter files:\n- {en_params}\n- {fi_params}")
else:
    print("Not creating new example files.")

Not creating new example files.


In [10]:
# Check parameter validation
print("Parameter Validation:")
print("-" * 50)
is_valid, warnings, errors = handler.validate()
if warnings:
    print("\nWarnings:")
    for warning in warnings:
        print(f"- {warning}")
if not is_valid:
    print("\nErrors:")
    for error in errors:
        print(f"- {error}")
else:
    print("\nParameters validated successfully!")

Parameter Validation:
--------------------------------------------------

Parameters validated successfully!


In [11]:
# 2. Test single language analysis
print("\nSingle Language Analysis:")
print("-" * 50)

options_en = AnalysisOptions(
    show_confidence=True,
    show_evidence=True,
    show_keywords=True,
    show_raw_data=True,
    debug_mode=True,
    language="en"  # Explicitly set language
)

# Analyze English text
text = example_texts["English Technical"]
results_en = await analyze_keywords(text, options_en)



2024-11-18 12:27:41,566 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-18 12:27:41,569 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:27:41,569 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-18 12:27:41,572 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:41,572 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer



Single Language Analysis:
--------------------------------------------------


2024-11-18 12:27:41,581 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_confidence=True, show_evidence=True, show_keywords=True, show_raw_data=True, debug_mode=True, language='en', parameter_file=None)
2024-11-18 12:27:41,583 - src.core.language_processing.factory - DEBUG - Using default configuration
2024-11-18 12:27:41,584 - src.core.language_processing.factory - DEBUG - Creating en processor
2024-11-18 12:27:41,587 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:27:41,587 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-18 12:27:41,590 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:27:41,590 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:


Keywords Found:
  • cloud                [████████████████████] (1.00)
  • migration            [████████████████████] (1.00)
  • scalability          [████████████████████] (1.00)
  • pipeline             [████████████████░░░░] (0.82)
  • streamlined          [███████████████░░░░░] (0.78)
  • project              [███████████████░░░░░] (0.76)
  • DevOps               [███████████████░░░░░] (0.76)
  • deployment           [██████████████░░░░░░] (0.72)
  • costs                [█████████████░░░░░░░] (0.66)
  • practices            [████████████░░░░░░░░] (0.62)

Debug Information:
--------------------
{
  "keywords": [
    {
      "keyword": "cloud",
      "score": 1.0,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "keyword": "migration",
      "score": 1.0,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "keyword": "scalability",
      "score": 1.0,
      "domain": "technical",
      "compound_parts": null
    },
    {
      "

In [12]:
# Test Finnish keyword analysis
print("\nTest Finnish keyword analysis:")
print("-" * 50)

options_fi = AnalysisOptions(
    show_confidence=True,
    show_evidence=True,
    debug_mode=True,
    language="fi"  # Explicitly set language
)

# Analyze Finnish text with auto-detection
text = example_texts["Finnish Technical"]
# results_fi = await analyze_text(text, options_fi)
keywords_fi = await analyze_keywords(text, options_fi)


# # 4. Test batch analysis with mixed languages
# print("\nBatch Analysis with Mixed Languages:")
# print("-" * 50)

# batch_results = {}
# for name, text in example_texts.items():
#     print(f"\nAnalyzing {name}:")
#     results = await analyze_text(text, options_auto)
#     batch_results[name] = results

# # 5. Test Excel file processing
# from src.nb_helpers.analyzers import analyze_excel_content

# # Create a test DataFrame
# import pandas as pd
# df = pd.DataFrame({
#     "content": example_texts.values(),
#     "type": [name.split()[0] for name in example_texts.keys()],  # "English" or "Finnish"
# })

# # Save to temporary Excel file
# temp_excel = "temp_test_content.xlsx"
# df.to_excel(temp_excel, index=False)

# print("\nExcel File Analysis:")
# print("-" * 50)

# await analyze_excel_content(
#     input_file=temp_excel,
#     output_file="analysis_results",
#     content_column="content",
#     parameter_file="parameters_en.xlsx",  # Use our parameter file
#     language_column="type"  # Use type column for language
# )

# # Clean up temporary file
# os.remove(temp_excel)

# # 6. Compare analysis results
# print("\nAnalysis Results Comparison:")
# print("-" * 50)

# def print_analysis_summary(results: dict, name: str):
#     print(f"\n{name}:")
#     if "keywords" in results:
#         keywords = results["keywords"].get("keywords", [])
#         print(f"Keywords found: {len(keywords)}")
#         for kw in keywords[:3]:  # Show top 3 keywords
#             print(f"- {kw.keyword}: {kw.score:.2f}")
    
#     if "themes" in results:
#         themes = results["themes"].get("themes", [])
#         print(f"Themes found: {len(themes)}")
#         for theme in themes[:2]:  # Show top 2 themes
#             print(f"- {theme.name}: {theme.confidence:.2f}")
            
#     if "categories" in results:
#         categories = results["categories"].get("categories", [])
#         print(f"Categories found: {len(categories)}")
#         for cat in categories[:2]:  # Show top 2 categories
#             print(f"- {cat.name}: {cat.confidence:.2f}")

# for name, results in batch_results.items():
#     print_analysis_summary(results, name)


2024-11-18 12:14:21,075 - src.nb_helpers.analyzers - DEBUG - Starting keyword analysis
2024-11-18 12:14:21,078 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:192] - Initialized FileUtils with log level: INFO
2024-11-18 12:14:21,078 - src.utils.FileUtils.file_utils - DEBUG - Initialized FileUtils with log level: INFO
2024-11-18 12:14:21,082 - src.utils.FileUtils.file_utils - DEBUG [file_utils.py:198] - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:14:21,082 - src.utils.FileUtils.file_utils - DEBUG - Project root: c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer
2024-11-18 12:14:21,088 - src.nb_helpers.analyzers - DEBUG - Initialized TextAnalyzer with options: AnalysisOptions(show_confidence=True, show_evidence=True, show_keywords=True, show_raw_data=True, debug_mode=True, language='fi', par


Test Finnish keyword analysis:
--------------------------------------------------


2024-11-18 12:14:21,148 - src.core.language_processing.finnish - INFO - Loaded 747 stopwords from c:\Users\tja\OneDrive - Rastor-instituutti ry\Tiedostot\Rastor-instituutti\kehittäminen\analytiikka\repos\semantic-text-analyzer\data\configurations\stop_words\fi.txt
2024-11-18 12:14:21,161 - src.core.language_processing.finnish - DEBUG - Added 27 technical stopwords
2024-11-18 12:14:21,164 - src.core.language_processing.finnish - DEBUG - Total Finnish stopwords: 774
2024-11-18 12:14:21,166 - src.core.language_processing.finnish - INFO - Detected platform: win32
2024-11-18 12:14:21,169 - src.core.language_processing.finnish - DEBUG - Could not initialize Voikko using system libraries: Could not find module 'libvoikko-1.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Exception ignored in: <function Voikko.__del__ at 0x0000011D69E375E0>
Traceback (most recent call last):
  File "c:\Users\tja\AppData\Local\miniconda3\envs\semantic-analyzer\lib\site-package


Debug Information:
--------------------
{
  "error": "__init__() got an unexpected keyword argument 'language_processor'",
  "success": false
}


In [13]:
# Run environment verification
# from src.nb_helpers.environment import verify_environment
# verify_environment()

In [14]:
# Test logging
logger = logging.getLogger("src.analyzers.keyword_analyzer")
logger.debug("Testing keyword analyzer logging")

2024-11-18 12:14:21,524 - src.analyzers.keyword_analyzer - DEBUG - Testing keyword analyzer logging


NOTE: following are not working with the new parameter handling model

In [ ]:
# Example texts in different languages
example_texts = {
    "English Technical": """
        The cloud migration project improved system scalability while reducing costs.
        New DevOps practices streamlined the deployment pipeline significantly.
    """,
    
    "Finnish Technical": """
        Pilvipalveluihin siirtyminen paransi järjestelmän skaalautuvuutta ja vähensi kustannuksia.
        Uudet DevOps-käytännöt tehostivat merkittävästi käyttöönottoprosessia.
    """
}

# Analyze with automatic language detection
for name, text in example_texts.items():
    print(f"\nAnalyzing {name}:")
    results = await analyze_text(text, options)

# Example with specific language and parameters
fi_options = AnalysisOptions(
    show_confidence=True,
    show_evidence=True,
    debug_mode=True,
    language="fi",
    parameter_file="finnish_params.yaml"
)

# Analyze Finnish text with specific parameters
fi_results = await analyze_text(example_texts["Finnish Technical"], fi_options)

# Batch process Excel file with language detection
await analyze_excel_content(
    input_file="multilingual_texts.xlsx",
    output_file="analysis_results",
    content_column="content",
    parameter_file="analysis_params.yaml",
    language_column="language"  # Optional column specifying language
)

<!-- ## Analysis Functions

### Single Analysis with Debug Output
Run detailed analysis for a single text: -->


In [8]:
example_texts = {
    "Business Analysis": """
        Q3 revenue increased by 15% with strong growth in enterprise sales.
        Customer retention improved while acquisition costs decreased.
        New market expansion initiatives are showing positive early results.
    """,
    
    "Technical Content": """
        The application uses microservices architecture with containerized deployments.
        Data processing pipeline incorporates machine learning models for prediction.
        System monitoring ensures high availability and performance metrics.
    """,
    
    "Mixed Content": """
        The IT department's cloud migration project reduced infrastructure costs by 25%.
        DevOps implementation improved deployment frequency while maintaining quality.
        Monthly recurring revenue from SaaS products grew steadily.
    """,
    "koulutus":
    """
        Verkko-oppimisalusta sisältää interaktiivisia moduuleja ja oman tahdin edistymisen seurannan. 
        Virtuaaliluokat mahdollistavat reaaliaikaisen yhteistyön opiskelijoiden ja ohjaajien välillä. 
        Digitaaliset arviointityökalut antavat välitöntä palautetta oppimistuloksista.
    """,
    "tekninen":
    """
        Koneoppimismalleja koulutetaan suurilla datajoukolla tunnistamaan kaavoja. 
        Neuroverkon arkkitehtuuri sisältää useita kerroksia piirteiden erottamiseen. 
        Datan esikäsittely ja piirteiden suunnittelu ovat keskeisiä vaiheita prosessissa.

    """
}

In [ ]:
# # New imports
# from src.core.language_parameters import LanguageParameterManager

# # Initialize parameter manager
# param_manager = LanguageParameterManager()

# # Example analysis with automatic language detection
# text_en = "Cloud computing enables scalable infrastructure deployment."
# text_fi = "Pilvipalvelut mahdollistavat skaalautuvan infrastruktuurin käyttöönoton."

# # Analyze with automatic language detection and default parameters
# async def analyze_text_with_language(text: str, parameter_file: Optional[str] = None):
#     """Analyze text with automatic language handling."""
#     # Get language-specific parameters
#     params = param_manager.get_parameters(text, parameter_file)
    
#     # Create analyzers with parameters
#     keyword_analyzer = KeywordAnalyzer(config=params.dict())
#     theme_analyzer = ThemeAnalyzer(config=params.dict())
#     category_analyzer = CategoryAnalyzer(config=params.dict())
    
#     # Run analysis
#     results = {
#         "keywords": await keyword_analyzer.analyze(text),
#         "themes": await theme_analyzer.analyze(text),
#         "categories": await category_analyzer.analyze(text)
#     }
    
#     return results

# # Example with Excel parameters
# async def analyze_batch_with_excel_params(texts: List[str], excel_params: str):
#     """Analyze texts using parameters from Excel."""
#     # Load language-specific parameters
#     params_by_lang = param_manager.load_excel_parameters(excel_params)
    
#     results = []
#     for text in texts:
#         # Detect language
#         lang = param_manager.detect_language(text)
#         # Get parameters for language
#         params = params_by_lang.get(lang, param_manager.get_parameters(text))
        
#         # Create analyzer with language-specific parameters
#         analyzer = KeywordAnalyzer(config=params.dict())
#         result = await analyzer.analyze(text)
#         results.append(result)
    
#     return results

# # Example usage:
# # With default parameters
# results_en = await analyze_text_with_language(text_en)

# # With parameter file
# results_fi = await analyze_text_with_language(text_fi, "finnish_params.yaml")

# # With Excel parameters
# texts = [text_en, text_fi]
# batch_results = await analyze_batch_with_excel_params(texts, "analysis_params.xlsx")

In [9]:
# text = example_texts["Mixed Content"]
# text = example_texts["koulutussisältö"]
# Debug specific analyzer

# Example usage
text = example_texts["Mixed Content"]

In [12]:
await analyze_keywords(text, options=options)


NameError: name 'options' is not defined

In [ ]:
await analyze_themes(text, options=options)



Debug Theme Analysis

Input Text:
--------------------
The IT department's cloud migration project reduced infrastructure costs by 25%.
        DevOps implementation improved deployment frequency while maintaining quality.
        Monthly recurring revenue from SaaS products grew steadily.

Running Analysis...
--------------------


In [ ]:
await analyze_categories(text, options=options)



Debug Category Analysis

Input Text:
--------------------
The IT department's cloud migration project reduced infrastructure costs by 25%.
        DevOps implementation improved deployment frequency while maintaining quality.
        Monthly recurring revenue from SaaS products grew steadily.

Running Analysis...
--------------------

Categories Found:

  • technical
    Confidence: [█████████████████░░░] (0.85)
    The text discusses a cloud migration project and DevOps implementation, both of which are technical processes related to software development and system management.
    Evidence:
      - cloud migration project
      - DevOps implementation
      - improved deployment frequency

  • business
    Confidence: [███████████████░░░░░] (0.75)
    The text mentions reduced infrastructure costs and growth in monthly recurring revenue, which are key indicators of business performance and financial health.
    Evidence:
      - reduced infrastructure costs by 25%
      - monthly rec

CategoryOutput(language='en', error=None, success=True, categories=[CategoryInfo(name='technical', confidence=0.85, explanation='The text discusses a cloud migration project and DevOps implementation, both of which are technical processes related to software development and system management.', evidence=['cloud migration project', 'DevOps implementation', 'improved deployment frequency'], themes=['cloud computing', 'DevOps', 'infrastructure management']), CategoryInfo(name='business', confidence=0.75, explanation='The text mentions reduced infrastructure costs and growth in monthly recurring revenue, which are key indicators of business performance and financial health.', evidence=['reduced infrastructure costs by 25%', 'monthly recurring revenue from SaaS products grew steadily'], themes=['cost reduction', 'revenue growth', 'SaaS business model'])], explanations={'technical': 'The text discusses a cloud migration project and DevOps implementation, both of which are technical processes

In [ ]:
# Or run full pipeline with debug info
await debug_full_pipeline(text)


### Batch Processing from Excel
Process multiple texts from Excel file:


In [ ]:
await analyze_excel_content(
    input_file="test_content.xlsx",  # Input Excel file path
    output_file="analysis_results",  # Output filename (without extension)
    content_column="content"         # Column containing text to analyze
)


## Parameters
- Configure analyzers using parameter files
- Control output detail with DebugOptions
- Set logging level for verbosity control

## Example Outputs
The analysis provides:
- Keywords with confidence scores
- Theme identification and descriptions
- Category classification with evidence
- Confidence visualizations with Unicode bars

## Notes
- Set logging level to WARNING to minimize output
- Use debug functions for detailed analysis inspection
- Excel output combines all analysis types